In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import transforms
import os
import shutil

In [2]:
img_dir = r'./data/dataset2'

In [3]:
species = ['cloudy', 'rain', 'shine', 'sunrise']

In [4]:
base_dir = r'./data/4weather'
train_dir = os.path.join(base_dir , 'train')
test_dir = os.path.join(base_dir , 'test')    

In [5]:
if not os.path.isdir(base_dir):
    os.mkdir(base_dir)
    train_dir = os.path.join(base_dir , 'train')
    test_dir = os.path.join(base_dir , 'test')    
    os.mkdir(train_dir)
    os.mkdir(test_dir)
    for sta_dir in ['train', 'test']:     
        for spec in species:
            os.mkdir(os.path.join(base_dir, sta_dir, spec))

In [6]:
for i, img_name in enumerate(os.listdir(img_dir)):
    for spec in species:
        if spec in img_name:
            if i%5 == 0:
                s = os.path.join(img_dir, img_name)
                d = os.path.join(base_dir, 'test', spec, img_name)
            else:
                s = os.path.join(img_dir, img_name)
                d = os.path.join(base_dir, 'train', spec, img_name)
            shutil.copyfile(s, d)

In [7]:
for sta_dir in ['train', 'test']:     
    for spec in species:
        print(sta_dir, spec, len(os.listdir(os.path.join(base_dir, sta_dir, spec))))

train cloudy 253
train rain 173
train shine 206
train sunrise 269
test cloudy 49
test rain 42
test shine 47
test sunrise 88


In [8]:
transform = transforms.Compose([
    transforms.Resize((96, 96)),
    transforms.ToTensor(), ##归一化零到1
    transforms.Normalize(mean=[.5, .5, .5], std=[.5, .5, .5])##标准化到-1到1之间
])

In [9]:
train_ds =  torchvision.datasets.ImageFolder(
        train_dir,
        transform=transform
)

In [10]:
test_ds =  torchvision.datasets.ImageFolder(
           test_dir,
           transform=transform
    )

In [11]:
train_ds.classes

['cloudy', 'rain', 'shine', 'sunrise']

In [12]:
train_ds.class_to_idx

{'cloudy': 0, 'rain': 1, 'shine': 2, 'sunrise': 3}

In [13]:
len(train_ds)

901

In [14]:
len(train_ds) + len(test_ds)

1127

In [15]:
len(test_ds)
BTACH_SIZE = 32

In [16]:
train_dl = torch.utils.data.DataLoader(
                            train_ds,
                            batch_size=BTACH_SIZE,
                            shuffle=True
)


In [17]:
test_dl = torch.utils.data.DataLoader(
                            test_ds,
                            batch_size=BTACH_SIZE,
)

In [18]:
imgs, labels = next(iter(train_dl))

In [19]:
imgs.shape

torch.Size([32, 3, 96, 96])

In [20]:
model = torchvision.models.vgg16(pretrained=True)

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [21]:
print(model)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [22]:
for param in model.features.parameters():
    param.requires_grad = False

In [23]:
model.classifier[-1].out_features = 4

In [24]:
optim = torch.optim.Adam(model.classifier.parameters(), lr=0.0001)
loss_fn = nn.CrossEntropyLoss()

In [25]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu',index= 1)
model = model.to(device)

KeyboardInterrupt: 

In [ ]:
def fit(epoch, model, trainloader, testloader):
    correct = 0
    total = 0
    running_loss = 0
    model.train()
    for x, y in trainloader:
        x, y = x.to(device), y.to(device)
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        optim.zero_grad()
        loss.backward()
        optim.step()
        with torch.no_grad():
            y_pred = torch.argmax(y_pred, dim=1)
            correct += (y_pred == y).sum().item()
            total += y.size(0)
            running_loss += loss.item()
        
    epoch_loss = running_loss / len(trainloader.dataset)
    epoch_acc = correct / total
        
        
    test_correct = 0
    test_total = 0
    test_running_loss = 0 
    model.eval()
    with torch.no_grad():
        for x, y in testloader:
            x, y = x.to(device), y.to(device)
            y_pred = model(x)
            loss = loss_fn(y_pred, y)
            y_pred = torch.argmax(y_pred, dim=1)
            test_correct += (y_pred == y).sum().item()
            test_total += y.size(0)
            test_running_loss += loss.item()
    
    epoch_test_loss = test_running_loss / len(testloader.dataset)
    epoch_test_acc = test_correct / test_total
    
        
    print('epoch: ', epoch, 
          'loss： ', round(epoch_loss, 3),
          'accuracy:', round(epoch_acc, 3),
          'test_loss： ', round(epoch_test_loss, 3),
          'test_accuracy:', round(epoch_test_acc, 3)
             )
        
    return epoch_loss, epoch_acc, epoch_test_loss, epoch_test_acc

In [ ]:
epochs = 10

In [ ]:
train_loss = []
train_acc = []
test_loss = []
test_acc = []

for epoch in range(epochs):
    epoch_loss, epoch_acc, epoch_test_loss, epoch_test_acc = fit(epoch,
                                                                 model,
                                                                 train_dl,
                                                                 test_dl)
    train_loss.append(epoch_loss)
    train_acc.append(epoch_acc)
    test_loss.append(epoch_test_loss)
    test_acc.append(epoch_test_acc)

epoch:  0 loss：  0.058 accuracy: 0.681 test_loss：  0.007 test_accuracy: 0.929
epoch:  1 loss：  0.005 accuracy: 0.946 test_loss：  0.003 test_accuracy: 0.951
epoch:  2 loss：  0.004 accuracy: 0.958 test_loss：  0.002 test_accuracy: 0.969
epoch:  3 loss：  0.002 accuracy: 0.978 test_loss：  0.003 test_accuracy: 0.969
epoch:  4 loss：  0.001 accuracy: 0.992 test_loss：  0.002 test_accuracy: 0.978


KeyboardInterrupt: 

In [2]:
import torch
import torch.nn as nn
m = nn.AdaptiveAvgPool2d((7,7))
input = torch.randn(2, 64, 1, 1)
output = m(input)
print('nn.AdaptiveAvgPool2d((7,7)):',output.shape)


nn.AdaptiveAvgPool2d((7,7)): torch.Size([2, 64, 7, 7])
